## A intenção do projeto é criar um chatbot baseado em reviews de filmes para que se possa fazer perguntas e manter uma conversa livre

- link do banco de dados https://www.kaggle.com/Cornell-University/movie-dialog-corpus?select=movie_lines.tsv
- referências
>- https://shanebarker.com/blog/deep-learning-chatbot/
> -https://towardsdatascience.com/how-to-create-a-chatbot-with-python-deep-learning-in-less-than-an-hour-56a063bdfc44

In [2]:
!pip3 install gensim
!pip3 install tensorflow
!pip3 install keras

In [160]:
import pandas as pd
import re
import gensim
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from scipy.spatial import distance

### Opening movie reviews

In [4]:
messages = pd.read_csv('./chatdata/movie_lines.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [5]:
messages.head(10)

,0,1,2,3,4
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.
5,L924,u2,m0,CAMERON,Wow
6,L872,u0,m0,BIANCA,Okay -- you're gonna need to learn how to lie.
7,L871,u2,m0,CAMERON,No
8,"""L870",u0,m0,BIANCA,I'm kidding. You know how sometimes you just ...
9,L869,u0,m0,BIANCA,Like my fear of wearing pastels?


### Cleaning the index of the conversations

In [6]:
messages.columns = ['msg_line', 'user1_id', 'movie_id', 'user_name', 'msg']

In [7]:
def remove_char(txt):
    return re.sub('[^0-9]','', txt)

In [8]:
messages['msg_line_clean'] = [remove_char(msg) for msg in messages['msg_line']]

In [9]:
messages.head(10)

,msg_line,user1_id,movie_id,user_name,msg,msg_line_clean
0,L1045,u0,m0,BIANCA,They do not!,1045
1,L1044,u2,m0,CAMERON,They do to!,1044
2,L985,u0,m0,BIANCA,I hope so.,985
3,L984,u2,m0,CAMERON,She okay?,984
4,L925,u0,m0,BIANCA,Let's go.,925
5,L924,u2,m0,CAMERON,Wow,924
6,L872,u0,m0,BIANCA,Okay -- you're gonna need to learn how to lie.,872
7,L871,u2,m0,CAMERON,No,871
8,"""L870",u0,m0,BIANCA,I'm kidding. You know how sometimes you just ...,870
9,L869,u0,m0,BIANCA,Like my fear of wearing pastels?,869


In [10]:
messages['msg_line_clean'] = pd.to_numeric(messages['msg_line_clean'])

In [11]:
messages = messages.set_index('msg_line_clean')

In [12]:
messages.head(10)

,msg_line,user1_id,movie_id,user_name,msg
msg_line_clean,,,,,
1045,L1045,u0,m0,BIANCA,They do not!
1044,L1044,u2,m0,CAMERON,They do to!
985,L985,u0,m0,BIANCA,I hope so.
984,L984,u2,m0,CAMERON,She okay?
925,L925,u0,m0,BIANCA,Let's go.
924,L924,u2,m0,CAMERON,Wow
872,L872,u0,m0,BIANCA,Okay -- you're gonna need to learn how to lie.
871,L871,u2,m0,CAMERON,No
870,"""L870",u0,m0,BIANCA,I'm kidding. You know how sometimes you just ...


### Opening conversation sequence

In [13]:
conv_seq = pd.read_csv('./chatdata/movie_conversations.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [14]:
conv_seq.head(10)

,0,1,2,3
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']
5,u0,u2,m0,['L271' 'L272' 'L273' 'L274' 'L275']
6,u0,u2,m0,['L276' 'L277']
7,u0,u2,m0,['L280' 'L281']
8,u0,u2,m0,['L363' 'L364']
9,u0,u2,m0,['L365' 'L366']


In [15]:
conv_seq.columns = ['user1_id', 'user2_id', 'movie_id', 'sequence']

In [16]:
conv_seq.head(10)

,user1_id,user2_id,movie_id,sequence
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']
5,u0,u2,m0,['L271' 'L272' 'L273' 'L274' 'L275']
6,u0,u2,m0,['L276' 'L277']
7,u0,u2,m0,['L280' 'L281']
8,u0,u2,m0,['L363' 'L364']
9,u0,u2,m0,['L365' 'L366']


### Build conversation

In [17]:
def split_conversation(txt):
    txt_alt = txt.split(' ')
    return txt_alt

In [18]:
def seq_to_list(seq):
    seq_list = [remove_char(s) for s in seq]
    return seq_list

In [19]:
messages['msg_2'] = '-'

In [20]:
def link_conversations(seq_list, df, filter1, filter2):
    i = 0
    while i in range(len(seq_list)):
        if i+1 < len(seq_list):
            next_msg = df.loc[int(seq_list[i+1]), filter1]
            #print(str(i))
            #print(next_msg)
            #print(seq_list[i])
            df.at[int(seq_list[i]), filter2] = next_msg
        i+=1

In [21]:
link_conversations(['194', '195', '196', '197'], messages, 'msg', 'msg_2')

In [22]:
messages.loc[195, 'msg']

"Well I thought we'd start with pronunciation if that's okay with you."

In [23]:
for conv in conv_seq['sequence']:
    #print(conv)
    seq = split_conversation(conv)
    #print(seq)
    #txt_alt = remove_char(txt_alt)      
    txt_alt = [remove_char(s) for s in seq]
    #print(txt_alt)
    link_conversations(txt_alt, messages, 'msg', 'msg_2')

In [24]:
messages.loc[194, 'msg_2']

"Well I thought we'd start with pronunciation if that's okay with you."

In [25]:
messages.head(30)

,msg_line,user1_id,movie_id,user_name,msg,msg_2
msg_line_clean,,,,,,
1045,L1045,u0,m0,BIANCA,They do not!,-
1044,L1044,u2,m0,CAMERON,They do to!,They do not!
985,L985,u0,m0,BIANCA,I hope so.,-
984,L984,u2,m0,CAMERON,She okay?,I hope so.
925,L925,u0,m0,BIANCA,Let's go.,-
924,L924,u2,m0,CAMERON,Wow,Let's go.
872,L872,u0,m0,BIANCA,Okay -- you're gonna need to learn how to lie.,-
871,L871,u2,m0,CAMERON,No,Okay -- you're gonna need to learn how to lie.
870,"""L870",u0,m0,BIANCA,I'm kidding. You know how sometimes you just ...,No


### Processing for deep learning

In [237]:
#keeping the just the pair of messages
df = messages[messages['msg_2'] != '-']

In [27]:
df.head(10)

,msg_line,user1_id,movie_id,user_name,msg,msg_2
msg_line_clean,,,,,,
1044,L1044,u2,m0,CAMERON,They do to!,They do not!
984,L984,u2,m0,CAMERON,She okay?,I hope so.
924,L924,u2,m0,CAMERON,Wow,Let's go.
871,L871,u2,m0,CAMERON,No,Okay -- you're gonna need to learn how to lie.
870,"""L870",u0,m0,BIANCA,I'm kidding. You know how sometimes you just ...,No
868,"""L868",u2,m0,CAMERON,"The """"real you"""".""",Like my fear of wearing pastels?
867,L867,u0,m0,BIANCA,What good stuff?,"The """"real you""""."""
866,L866,u2,m0,CAMERON,I figured you'd get to the good stuff eventually.,What good stuff?
864,L864,u0,m0,BIANCA,Me. This endless ...blonde babble. I'm like b...,Thank God! If I had to hear one more story ab...


In [28]:
def pre_processamento_texto(corpus):
    #tokenizacao
    corpus_alt = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    #capitalizacao
    corpus_alt = [t.lower() for t in corpus_alt]
    #lammatization
    #sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    
    #stemming
    #TODO
    
    #remover stopwords
    #stopwords_ = stopwords.words("portuguese")
    #corpus_alt = [t for t in corpus_alt if t not in stopwords_]
    #remover numero
    #corpus_alt = [re.sub(r"\d","",t) for t in corpus_alt]
    #remover pontuacoes
    #corpus_alt = [t for t in corpus_alt if t not in string.punctuation]

    return corpus_alt

In [29]:
df['msg_proc'] = [pre_processamento_texto(str(txt)) for txt in df['msg']]

<ipython-input-29-761e6edbaf18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['msg_proc'] = [pre_processamento_texto(str(txt)) for txt in df['msg']]


In [30]:
df['msg_2_proc'] = [pre_processamento_texto(str(txt)) for txt in df['msg_2']]

<ipython-input-30-ec576d7df925>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['msg_2_proc'] = [pre_processamento_texto(str(txt)) for txt in df['msg_2']]


In [31]:
df

,msg_line,user1_id,movie_id,user_name,msg,msg_2,msg_proc,msg_2_proc
msg_line_clean,,,,,,,,
1044,L1044,u2,m0,CAMERON,They do to!,They do not!,"[they, do, to, !]","[they, do, not, !]"
984,L984,u2,m0,CAMERON,She okay?,I hope so.,"[she, okay, ?]","[i, hope, so, .]"
924,L924,u2,m0,CAMERON,Wow,Let's go.,[wow],"[let's, go, .]"
871,L871,u2,m0,CAMERON,No,Okay -- you're gonna need to learn how to lie.,[no],"[okay, -, -, you're, gonna, need, to, learn, h..."
870,"""L870",u0,m0,BIANCA,I'm kidding. You know how sometimes you just ...,No,"[i'm, kidding, ., you, know, how, sometimes, y...",[no]
...,...,...,...,...,...,...,...,...
666520,L666520,u9034,m616,VEREKER,Well I assure you Sir I have no desire to crea...,And I assure you you do not In fact I'd be obl...,"[well, i, assure, you, sir, i, have, no, desir...","[and, i, assure, you, you, do, not, in, fact, ..."
666371,L666371,u9030,m616,DURNFORD,Lord Chelmsford seems to want me to stay back ...,I think Chelmsford wants a good man on the bor...,"[lord, chelmsford, seems, to, want, me, to, st...","[i, think, chelmsford, wants, a, good, man, on..."
666370,L666370,u9034,m616,VEREKER,I'm to take the Sikali with the main column to...,Lord Chelmsford seems to want me to stay back ...,"[i'm, to, take, the, sikali, with, the, main, ...","[lord, chelmsford, seems, to, want, me, to, st..."


In [32]:
df['msg'] = df['msg'].astype(str)

<ipython-input-32-bb7fe2566607>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['msg'] = df['msg'].astype(str)


In [33]:
df['msg_2'] = df['msg_2'].astype(str)

<ipython-input-33-f263fbba459d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['msg_2'] = df['msg_2'].astype(str)


In [65]:
# primeiro criamos o objeto
vect_bag = CountVectorizer(binary=True) #se binary = False -> ocorre a contagem da frequência em que a palavra aparece
vect_bow = vect_bag.fit_transform(df['msg'])

In [66]:
vect_bag.get_feature_names()

['00',
 '000',
 '007',
 '0088',
 '009843',
 '010',
 '0100',
 '0130',
 '015',
 '01766',
 '0199',
 '02pm',
 '03',
 '0300',
 '04',
 '05',
 '0500',
 '06',
 '0630',
 '07410',
 '0800',
 '0821',
 '09',
 '0900',
 '0972',
 '0h',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '1002',
 '1009',
 '101',
 '1012',
 '101st',
 '104',
 '105',
 '106',
 '1060',
 '108',
 '109',
 '1098',
 '10cc',
 '10th',
 '11',
 '110',
 '1138',
 '114',
 '115',
 '119',
 '11speak',
 '11th',
 '12',
 '120',
 '1200',
 '12000',
 '120000',
 '1208',
 '122',
 '124',
 '125',
 '125th',
 '127',
 '1274',
 '1294',
 '12th',
 '13',
 '130',
 '131',
 '132',
 '1320',
 '1322',
 '133',
 '134',
 '135',
 '137',
 '138',
 '139',
 '13th',
 '14',
 '140',
 '14000',
 '1412',
 '1425',
 '143',
 '1436',
 '1443',
 '145',
 '146',
 '147',
 '1490',
 '14b',
 '14th',
 '15',
 '150',
 '1500',
 '15000',
 '1505',
 '153',
 '1530',
 '154',
 '155',
 '1550',
 '156',
 '15th',
 '16',
 '160',
 '1600',
 '161',
 '1610',
 '1620',
 '167',
 '1672',
 '16m',
 '16mm',
 '16s',
 '1

In [135]:
print(vect_bow)

  (0, 38274)	1
  (0, 11321)	1
  (0, 38682)	1
  (1, 34042)	1
  (1, 26594)	1
  (2, 42470)	1
  (3, 25968)	1
  (4, 38682)	1
  (4, 20890)	1
  (4, 42769)	1
  (4, 21117)	1
  (4, 18398)	1
  (4, 35430)	1
  (4, 20614)	1
  (4, 3719)	1
  (4, 38328)	1
  (4, 28207)	1
  (4, 1918)	1
  (4, 11453)	1
  (4, 30413)	1
  (5, 42769)	1
  (5, 38190)	1
  (5, 30846)	1
  (6, 41799)	1
  (6, 16205)	1
  :	:
  (221612, 22531)	1
  (221612, 6870)	1
  (221612, 3535)	1
  (221613, 38682)	1
  (221613, 38190)	1
  (221613, 42247)	1
  (221613, 37631)	1
  (221613, 23012)	1
  (221613, 32189)	1
  (221613, 7843)	1
  (221613, 34549)	1
  (221614, 42781)	1
  (221614, 25074)	1
  (221614, 26855)	1
  (221614, 40787)	1
  (221615, 42769)	1
  (221615, 40683)	1
  (221615, 3767)	1
  (221615, 17492)	1
  (221615, 42089)	1
  (221615, 26531)	1
  (221615, 7806)	1
  (221615, 33602)	1
  (221615, 40787)	1
  (221615, 12014)	1


In [119]:
pd.DataFrame(vect_bow[0:10].todense(), columns=vect_bag.get_feature_names())

,00,000,007,0088,009843,010,0100,0130,015,01766,...,zurich,zus,zuul,zuzu,zwei,zwoelf,zydowski,zygmunt,zyprexa,zzzzzzzzzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [189]:
df_small = pd.DataFrame()

In [190]:
df_small['msg'] = df['msg'][0:3]

In [191]:
df_small['msg_2'] = df['msg_2'][0:3]

In [193]:
train_x = df_small['msg']
train_y = df_small['msg_2']

In [194]:
from keras.preprocessing.text import Tokenizer
# define documents
docs = train_x
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(docs)

In [195]:
# encode training data set
Xtrain = tokenizer.texts_to_matrix(train_x)
# encode training data set
Xtest = tokenizer.texts_to_matrix(train_y)

In [196]:
Xtrain.shape

(3, 7)

In [197]:
Xtest.shape

(3, 7)

In [198]:
print(distance.cosine(Xtrain[0], Xtest[0]))

0.18350341907227397


In [199]:
num_rows, num_cols = Xtrain.shape

In [221]:
import math

list_similarity = []
for i in range(num_rows):
    d = distance.cosine(Xtrain[i], Xtest[i])
    if math.isnan(d):
        d = 0.05
    list_similarity.append(d)
    i+=0

/home/douglas/.local/lib/python3.8/site-packages/scipy/spatial/distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [222]:
df_small['similarity'] = list_similarity

In [223]:
df_small

,msg,msg_2,similarity
msg_line_clean,,,
1044,They do to!,They do not!,0.183503
984,She okay?,I hope so.,0.050000
924,Wow,Let's go.,0.050000


In [224]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(2, input_dim=7, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

model.summary()

Model: "sequential_46"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_127 (Dense)            (None, 2)                 16        
_________________________________________________________________
dropout_82 (Dropout)         (None, 2)                 0         
_________________________________________________________________
dense_128 (Dense)            (None, 2)                 6         
_________________________________________________________________
dropout_83 (Dropout)         (None, 2)                 0         
_________________________________________________________________
dense_129 (Dense)            (None, 1)                 3         
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


In [225]:

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(Xtrain, df_small['similarity'], epochs=2, batch_size=3, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/2
1/1 [==============================] - 0s 4ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 0s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
model created


In [226]:
p = tokenizer.texts_to_matrix('what is the best movie?')

In [227]:
res = model.predict(np.array([p]))[0]

In [236]:
res

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [228]:
ERROR_THRESHOLD = 0.25
results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

In [229]:
# sort by strength of probability
results.sort(key=lambda x: x[1], reverse=True)

In [235]:
return_list = []
i = 0
for i in range(len(results)):
    #return_list.append({"intent": df_small['msg_2'][i], "probability": str(i)})
    print(results[i])

[0, array([1.], dtype=float32)]
[1, array([1.], dtype=float32)]
[2, array([1.], dtype=float32)]
[3, array([1.], dtype=float32)]
[4, array([1.], dtype=float32)]
[5, array([1.], dtype=float32)]
[6, array([1.], dtype=float32)]
[7, array([1.], dtype=float32)]
[8, array([1.], dtype=float32)]
[9, array([1.], dtype=float32)]
[10, array([1.], dtype=float32)]
[11, array([1.], dtype=float32)]
[12, array([1.], dtype=float32)]
[13, array([1.], dtype=float32)]
[14, array([1.], dtype=float32)]
[15, array([1.], dtype=float32)]
[16, array([1.], dtype=float32)]
[17, array([1.], dtype=float32)]
[18, array([1.], dtype=float32)]
[19, array([1.], dtype=float32)]
[20, array([1.], dtype=float32)]
[21, array([1.], dtype=float32)]
[22, array([1.], dtype=float32)]


#### Template model

In [123]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(2, input_dim=4, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

model.summary()

Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_115 (Dense)            (None, 2)                 10        
_________________________________________________________________
dropout_74 (Dropout)         (None, 2)                 0         
_________________________________________________________________
dense_116 (Dense)            (None, 2)                 6         
_________________________________________________________________
dropout_75 (Dropout)         (None, 2)                 0         
_________________________________________________________________
dense_117 (Dense)            (None, 1)                 3         
Total params: 19
Trainable params: 19
Non-trainable params: 0
_________________________________________________________________


In [124]:

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(([1,2,3,4],[1,2,3,4],[1,2,3,4]), [1,2,3], epochs=2, batch_size=3, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/2
1/1 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.3333
Epoch 2/2
1/1 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.3333
model created
